# Final Report: Question Predictor for StackOverflow

## Overview

StackOverflow is the largest online QA platform for programmers to ask, answer various questions and learn new knowledge and technique. By the end of August 2015, there are more than 10,000,000 questions posted on Stack Overflow. However, As the number of questions grows, we may find that there are more and more questions that focuses on the same topic or features, or actually have the same answers. We define these questions as duplicate questions. Duplicate questions will waste the machine resources and made question raiser to wait for a long time for a queston that has been answered already. Currently, Stack Overflow encourage users to manually mark any potential duplicate questions, which is not efficient and may have some bias. Here is an example of duplicate questions:

![](image01.jpg)

Stack Overflow **does not** have a standard method or label to distinguish duplicate questions from normal questions. However, Stack Overflow users usually have default measures to signify possible duplicate questions. We use three conditions to decide whether a question is a duplicate question or not:

1. Check where **[duplicate]** appears in the title.
2. Check whether [**This question already has an answer**] or  in the body of the questions.
3. Check whether [**Possible duplicate**] in the body of the questions.

A question satisfying any of the forementioned conditions will be classified as duplicate questions.

## Data Preparation

We got our data from [here](https://archive.org/details/stackexchange) [1].


The original date is in XML format. We need to keep only all questions(PostTypeId=1) and remove invalid questions(Id=-1). The following code is used to extract useful information from data. For duplicate questions, we also extract the id of the existing questions for further analysis.

#### The basic statistic of the data is listed in the following:

Total number of records: 32209819


Total number of valid questions: 12350818

Total number of duplicate questions: 48197


In [1]:
import re
import sys
import json


def data_parser():
    regex = re.compile('([a-z0-9]+)="([^"]+)"', re.I)
    dup_id = re.compile("stackoverflow.com/questions/(\d+)/")

    fp_all = open("all_questions", "w")
    fp_dup = open("dup_questions", "w")
    with open(sys.argv[1], "r") as fp:
        for line in fp:
            try:
                line = line.strip()
                post = dict(re.findall(regex, line))
                if "Id" not in post:
                    continue

                if "has already been answered" in post["Body"] or ("Title" in post and "[duplicate]" in post["Title"]):
                    post["dups"] = [int(num) for num in re.findall(dup_id, post["Body"])]
                    fp_dup.write(post["Id"] + "wcyz666SQL" + json.dumps(post) + "\n")

                fp_all.write(post["Id"] + "wcyz666SQL" + json.dumps(post) + "\n")
            except:
                pass

Then we load the data to **MySQL database** (deployed on Amazon AWS) for future retrival.

CREATE TABLE `all_questions` (`id` bigint(64), `text` text charset 'utf8mb4' collate utf8mb4_unicode_ci) ENGINE=MyISAM DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;

CREATE TABLE `dup_questions` (`id` bigint(64), `text` text charset 'utf8mb4' collate utf8mb4_unicode_ci) ENGINE=MyISAM DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;

load data local infile '/home/ubuntu/all_questions' into table all_questions character set 'utf8mb4' fields terminated by "wcyz666SQL";

load data local infile '/home/ubuntu/dup_questions' into table dup_questions character set 'utf8mb4' fields terminated by "wcyz666SQL";

create INDEX pds_index1 on all_questions(id);

create INDEX pds_index2 on dup_questions(id);




## Data Cleaning

In order to obtain duplicate pairs, we need to insert more questions into MySQL database. In the previous step, the questions that has been marked as **duplicate questions** will has links (enclosing question ID inside) to source questions, which we should also insert to form duplicate pairs. 

There are 1998 duplicate questions marked but have no external link to other questions. These questions will be eliminated in this step. Other questions will be iterated one-by-one and source questions will be selected from *all_questions* and then inserted to *dup_questions*. Related code is shown in the following:

In [2]:
import json
import MySQLdb

def data_clean():
    fp_dup = open("duped_questions", "w")
    cnx = MySQLdb.connect(user='root', password='123456',
                                  host='ec2-52-91-216-34.compute-1.amazonaws.com',
                                  database='pds_team')

    fp_dup = open("duped_questions", "w")
    cnx = MySQLdb.connect(user='root', password='',
                                  host='ec2-52-91-216-34.compute-1.amazonaws.com',
                                  database='pds_team')

    all_questions = MySQLdb.connect(user='root', password='',
                                  host='ec2-52-91-216-34.compute-1.amazonaws.com',
                                  database='pds_team')
    no_dup = []
    cnx.query("SELECT * FROM dup_questions")
    res = cnx.use_result()
    count = 0
    while True:
        result = res.fetch_row(maxrows=100)
        if len(result) == 0:
            break
        count += 100
        print str(count) + " items is completed"
        for id, post_json in result:
            try:
                post = json.loads(post_json)
                if len(post['dups']) == 0:
                    no_dup.append(id)
                else:
                    for dup_id in post["dups"]:
                        all_questions.query("SELECT * FROM all_questions WHERE id=" + str(dup_id))
                        all_res = all_questions.store_result()
                        for _, item in all_res.fetch_row():

                            fp_dup.write(str(dup_id) + "wcyz666SQL" + item + "\n")
            except:
                pass
    fp_dup.close()
    cnx.close()

## Feature Extraction

To evaluate the similarity between two questions, we applied different indicators as features, including title, questions body, question topic tags. Besides, we also applied the LDA model to evaluate the similarity on topics between the questions.


### Preprocess

We preprocess all the texts in the question title/content by removing punctuations, stemming, tokenizing etc.

In [3]:
def tokenize_with_preprocess(text):
    """
    Tokenize with preprocessing. Remove all punctuations and apply stemming on the tokens
    :param text: String
    :return: list of tokens
    """
    return map(__stemmer.stem, filter(lambda w: w not in stop,
                                        nltk.word_tokenize(re.sub(_punc_pattern, '', text.lower()))))

### Text similarity
We try both cosine and jacard similarity as the similarity indicator, based on tfidf/ token set respectively.

In [4]:
def jaccard_similarity(text1, text2):
    """
    :param text1, text2: list of tokens
    :return: float
    """
    return 1.0 - nltk.jaccard_distance(set(text1), set(text2))

def cosine_similarity(text1, text2):
    """
    :param text1, text2: list of tokens
    :return: float
    """
    tfidf = TfidfVectorizer().fit_transform(map(_SimilarityUtil._inverse, [text1, text2]))
    return 1 - cosine(tfidf[0].todense(), tfidf[1].todense())

### LDA model
We applied LDA on the train set to model the topics among each questions. During evaluating, we use the pretrained lda model to tranform the text and get a distribution(a vetor of topic_size) among all the topics. We compute the euclidean distance of the two distribution vector to get the feature.

As the size of the dataset is rather large(with over 10k samples and an average length of over 500 words), we failed to train the lda model based on all text materials. As a compromise, we choose the most common 1000 words(already stemmed and preprocessed by the function above) as the base dictionary in lda.

In [5]:
class LDA(object):
    def __init__(self, corpus, load=False, n_topic=3):
        self.words = np.array([t[0] for t in Counter(itertools.chain(*corpus)).most_common(1000)])
        X = np.array(map(self._transform, corpus))
        self._model = lda.LDA(n_topics=n_topic, random_state=0, n_iter=100)
        self._model.fit(X)
    
    def get_topic(self, tokens):
        return self._model.transform(self._transform(tokens))
    
    # more functions to go..............................

This snippet of code shows how we train the model. The body of all questions in the train_set are fed to the model.

In [6]:
def build_lda_model(train_set):
    lda = LDA([q['body'] for q in train_set.values()], n_topic=20)
    lda.print_topic_top_words(n_top_words=8)
    return lda

### Combine them together
For each question, it has a list of attributes. With experiment, we decide to choose "title", "body", "tag" and the "topic" inferred by the lda model above as separate features. As a result, in the final implemenation of our model, the similarity(the possibility of duplicate) of each pair of questions are determined by a 4-dimension vector.

Here is how we extract the information of each question from the raw dataset.

In [7]:
# Choose preprocessing for tokenize
tokenize = tokenize_with_preprocess

def _stripe_tag(tags):
    return sorted(tags.replace('&lt', '').replace('&gt;', ' ').split())

def preprocess(q_set):
    return {j['Id']: {'id': j['Id'],
                      'title': tokenize(j['Title']),
                      'body': tokenize(j['Body']),
                      'tags': _stripe_tag(j['Tags'])} for j in q_set}

As described above in this section, we are able to build a feature vector for each pair of questions in the dataset.

In [8]:
def build_feature(q1, q2, lda):
    """
    :param q1, q2: Pre-processed questions. A dict of all required attributes
    :param lda: Pre-trained lda model
    :return: A 4-d array of feature 
    """
    # Title similarity
    x0 = SimilarityUtil.cosine_similarity(q1['title'], q2['title'])
    x1 = SimilarityUtil.cosine_similarity(q1['body'], q2['body'])
    # x2 = SimilarityUtil.jaccard_similarity(q1['tags'], q2['tags'])
    x2 = SimilarityUtil.cosine_similarity(q1['tags'], q2['tags'])
    x3 = np.linalg.norm(lda.get_topic(q1['body']) - lda.get_topic(q2['body']))
    return np.array([x0, x1, x2, x3])

## Train & Validate
An interesting thing in this problem is that the actual train/test sample is not a single question itself, but the relationship between to questions. It is feasible to build and validate our model by spliting all the relationships(pairs) but we found some problems.

1. With around 10k questions, the algorithm will yield over 10^8 pair which is a disaster for the model to build.
2. Even if we have a powerful machine, the model will be extremly biased as only 1/10k of the pairs are positive samples.
3. The model cannot be applied to discovering duplicate questions easily.

As a result, we decided to find another to build the model and it will be even better if the model cna be applied in to real world use cases.

### Training and Test Set
Considering the real world use case, we split the question set into 70% vs 30% as old questions and new questions. Old questions are regarded as those already been posted in the forum and are used in the training phase to build our model. New questions are known as the questions that are posted after the build of the model and thus we will use the model to find/validate the duplicate questions within new question sets.

Later, our model will be validate both on if a new question is duplicate from an old question and if a new question is duplicate from another new question, which matches the real world use case.

In [9]:
def load():
    all_questions = QuestionSet('ec2-52-91-216-34.compute-1.amazonaws.com', 'root', '123456', 'pds_team')
    qids = all_questions.keys()
    random.shuffle(qids)
    split_size = int(len(qids) * 0.7)

    print "split train test sets"
    train_set = preprocess(all_questions[qid] for qid in qids[:split_size])
    test_set = preprocess(all_questions[qid] for qid in qids[split_size:])

The preparation of training set is quite straight forward. We picked all the duplicated question pairs in the old questions as the training data. To keep the training dataset unbias, we randomly pick pairs with comparable amount as the negative sample.

In [10]:
def prepare_train_data(all_questions, train_set, lda):
    X = []
    y = []
    for q in train_set:
        for dup in all_questions.get_dup(q):
            if dup in train_set:
                X.append(build_feature(train_set[q], train_set[dup], lda))
                y.append(1)
    print 'Generating random negative samples'
    # Generate negative samples as the same scale
    size_positive = len(y)
    q_set = train_set.keys()
    for _ in xrange(size_positive):
        q1 = random.choice(q_set)
        q2 = random.choice(q_set)
        X.append(build_feature(train_set[q1], train_set[q2], lda))
        y.append(0 if q2 not in all_questions.get_dup(q1) else 1)

    return np.array(X), np.array(y)

The preparation of test data is almost the same, except that we explicitly distinguish the relationship of old-new from new-new question pairs, as discussed above.

In [11]:
def prepare_validation_data(all_questions, train_set, test_set, lda):
    """
    :param all_questions:
    :param test_set:
    :param lda:
    :return: A tuple of four
     X_test_cross: feature matrix between newly found questions and old qustions
     y_test_cross: label array .................................................
     X_test_new:   feature matrix among newly found questions
     y_test_new:   label array ..............................
    """
    X_test_cross, y_test_cross = [], []
    X_test_new, y_test_new = [], []
    for q in test_set:
        for dup in all_questions.get_dup(q):
            if dup in test_set:
                # Among new questions
                X_test_new.append(build_feature(test_set[q], test_set[dup], lda))
                y_test_new.append(1)
            elif dup in train_set:
                X_test_cross.append(build_feature(test_set[q], train_set[dup], lda))
                y_test_cross.append(1)

    # Generate negative samples as the same scale
    print 'Generating random negative samples'
    size_positive_cross = len(y_test_cross)
    size_positive_new = len(y_test_new)
    q_set_train = train_set.keys()
    q_set_test = test_set.keys()

    for _ in xrange(20*size_positive_cross):
        q1 = random.choice(q_set_train)
        q2 = random.choice(q_set_test)
        X_test_cross.append(build_feature(train_set[q1], test_set[q2], lda))
        y_test_cross.append(0 if q2 not in all_questions.get_dup(q1) else 1)

    for _ in xrange(size_positive_new):
        q1 = random.choice(q_set_test)
        q2 = random.choice(q_set_test)
        X_test_new.append(build_feature(test_set[q1], test_set[q2], lda))
        y_test_new.append(0 if q2 not in all_questions.get_dup(q1) else 1)

    return tuple(map(np.array, (X_test_cross, y_test_cross, X_test_new, y_test_new)))

Finally we end up with `X_train` `y_train` as the training data and two pairs of test dataset.

In [12]:
def load():
    # .......................
    
    print 'loading training set'
    X_train, y_train = prepare_train_data(all_questions, train_set, lda)
    print 'loading test set'
    X_test_cross, y_test_cross, X_test_new, y_test_new = \
        prepare_validation_data(all_questions, train_set, test_set, lda)

## Evaluation

You might notice that in the above code snippet, even in the test phase, the negative pairs vs positive pairs are among 1:1, which is not true in real word. To further ensure the usability, we generate three test set with different negative sample weight of 1,5,20, which means in the validation dataset, the duplicate vs non-duplicate pairs are 1:1, 1:5, 1:20 respectively. We found that if we increase the weight further, say 40, 50, 100 etc, it won't affect the performance of our model significantly. As a result we use the performance of these three weight scale as the metrics of our model.

Since the feature vector only length 4, we choose the simple logistic regression which provides comparable performance as other more sophiscated ones but is much easier to be explained and understand in real word.

In [13]:
from sklearn import linear_model
logistic = linear_model.LogisticRegression(class_weight={0:5, 1:1})
lr = logistic.fit(X_train, y_train)

from sklearn.metrics import classification_report
print classification_report(lr.predict(X_test_cross), y_test_cross)
print classification_report(lr.predict(X_test_new), y_test_new)

NameError: name 'X_train' is not defined

             precision    recall  f1-score   support

          0       0.99      0.99      0.99    402188
          1       0.80      0.82      0.81     19597

avg / total       0.98      0.98      0.98    421785

             precision    recall  f1-score   support

          0       0.99      0.99      0.99    173388
          1       0.81      0.82      0.82      8514

avg / total       0.98      0.98      0.98    181902


We didn't do lots of param grid search work apart from simply set the class weight to add the penalty for true-negative errors(duplicate pairs predicted as non-duplicate) since our point is to discover all the potential duplicate questions. The most meaningful number we care about in the report is the recall rate of the duplicate questions(marked as 1).

## Result and Analysis

### Recall rate
With a 20 times negative weight, we are able to achieve slightly over 80% recall rate with a comparable precision rate.

![](result1.png)

### Coef of each feature

The coefficiency describes to what extent a feature contirbutes to the prediction. Still we found the similarity of body text and their topic is the determining feature.

In [ ]:
coef_ = {'title': 16.12967728, 'body': 19.89534911, 'tag': 7.91410006, 'topic': 19.8966333}

### Discover new unmarked duplicate pairs

Finally we will apply our model into reality. We randomly choose 10000 questions from *all_questions*, and use the classifier to see if there is any potential duplicate pairs that are not marked as duplicate pairs.

In [ ]:
def main_discover_new():
    
    warnings.filterwarnings("ignore")
    with open('final-data20.pkl', 'rb') as f:
        res = pickle.load(f)
    with open('all_questions.pkl', 'rb') as f:
        all = pickle.load(f)
        
    print "train data loaded"
    
    res = [np.nan_to_num(a) for a in res]

    X_train, y_train, _, _, _, _ = res
    logistic = linear_model.LogisticRegression(class_weight={0:1, 1:1})
    lr = logistic.fit(X_train, y_train)
    print "trained data ready"

    with open('lda.pkl', 'rb') as f:
        lda = pickle.load(f)
    print "lda data loaded"

    random_questions = get_random(10000, all)
    ids = list(random_questions.keys())
    for id_1 in ids:
        for id_2 in ids:
            if id_1 != id_2:
                try:
                    feature = build_feature(random_questions[id_1], random_questions[id_2], lda)
                    if lr.predict(feature)[0] == 1:
                        print id_1, id_2
                except:
                    pass

The running result is:

In [ ]:
11200069 1499950
11200069 14014974
11200069 1248100
11200069 14042658
11200069 635297
9976291 3622267
9976291 6006276
9976291 12203403
9976291 11597222
9976291 12489473
9976291 9461709
....

Unfortunatly, we can still find some mis-classified duplicate pairs. Even more unfortunatly, for instance, the first pair: The content of question 11200069 is 

![](image02.jpg)

And the second question 1499950 is 

![](image03.jpg)

We can see then have similar tags, title and topics and similar content, but they are talking about different tech details and should not be marked as duplicate.

This is obviously a limitation of our model. Even if two questions have the same tags, same topics and similar content, they are not necessarily to be duplicate, since two unique questions might be asked on exactly the same problem/code/api etc. and it is extremly difficult for algorithms to differentiate.

## Conclusion

In this project, we applied machine learning related technique to discover potential duplicate questions in Stackoverflow. We collect the raw XML-formatted data from Stackoverlow archive and stored the clean data into MySQL. Then We extract four features, namely, **title, topic, post and tag**, calculate the distance among them using LDA model and cosine/jaccard similarity. After that, we partition our dataset, and train our model using 70% of the original dataset using **logistic regression**. Finally we validate our model using the validation set, and tune some parameters to achieve the best precision.

## Reference

1. https://archive.org/details/stackexchange
2. Zhang Y, Lo D, Xia X et al. Multi-factor duplicate question detection in Stack Overflow. JOURNAL OF COMPUTER
SCIENCE AND TECHNOLOGY 30(5): 981–997 Sept. 2015. DOI 10.1007/s11390-015-1576-4

## Appendix A
Our full source code is available at: https://github.com/lumig242/DuplicationDetectionStackOverflow